In [1]:
import sys
#sys.path.insert(0, "../..")

from typing import Callable, Protocol, Dict, Optional, Iterator, List, Tuple

import gin
from pathlib import Path
import numpy as np
import math
from datetime import datetime

import torch
from torch import nn
import torch.optim as optim
from torch.optim import Optimizer
from torchinfo import summary
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# ------------------------------------------------------------------------------- #
## Alle imports benodigd voor de functie train_loop uit train_model.py 

import tensorflow as tf  # noqa: F401

# needed to make summarywriter load without error
from loguru import logger
from numpy import Inf
from ray import tune


from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from tqdm import tqdm

2022-12-23 15:15:15.245965: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-23 15:15:16.213809: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-23 15:15:16.213906: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-23 15:15:16.213917: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
## benodigde functie uitmake_dataset.py

@gin.configurable
def get_MNIST(  # noqa: N802
    data_dir: Path, batch_size: int
) -> Tuple[DataLoader, DataLoader]:

    training_data = datasets.FashionMNIST(
        root=data_dir,
        train=True,
        download=True,
        transform=ToTensor(),
    )

    test_data = datasets.FashionMNIST(
        root=data_dir,
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader

In [3]:
## Benodigde funties uit metrics.py

Tensor = torch.Tensor

class Metric:
    def __repr__(self) -> str:
        raise NotImplementedError

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        raise NotImplementedError

class Accuracy(Metric):
    def __repr__(self) -> str:
        return "Accuracy"

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        return (yhat.argmax(dim=1) == y).sum() / len(yhat)

class GenericModel(Protocol):
    train: Callable
    eval: Callable
    parameters: Callable

    def __call__(self, *args, **kwargs) -> torch.Tensor:
        pass


def count_parameters(model: GenericModel) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


## uit data_tools.py
def dir_add_timestamp(log_dir: Optional[Path] = None) -> Path:
    if log_dir is None:
        log_dir = Path(".")
    log_dir = Path(log_dir)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M")
    log_dir = log_dir / timestamp
    logger.info(f"Logging to {log_dir}")
    if not log_dir.exists():
        log_dir.mkdir(parents=True)
    return log_dir

In [4]:
## Definieren functie train_loop

@gin.configurable
def trainloop(
    epochs: int,
    model: GenericModel,
    optimizer: torch.optim.Optimizer,
    learning_rate: float,
    loss_fn: Callable,
    metrics: List[Metric],
    train_dataloader: Iterator,
    test_dataloader: Iterator,
    log_dir: Path,
    train_steps: int,
    eval_steps: int,
    patience: int = 10,
    factor: float = 0.9,
    tunewriter: bool = False,
) -> GenericModel:
    
    optimizer_: torch.optim.Optimizer = optimizer(
        model.parameters(), lr=learning_rate
    )  # type: ignore

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_,
        factor=factor,
        patience=patience,
    )

    if not tunewriter:
        log_dir = dir_add_timestamp(log_dir)
        writer = SummaryWriter(log_dir=log_dir)
        #write_gin(log_dir, gin.config_str())

        images, _ = next(iter(train_dataloader))
        if len(images.shape) == 4:
            grid = make_grid(images)
            writer.add_image("images", grid)
        writer.add_graph(model, images)

    for epoch in tqdm(range(epochs)):
        train_loss = trainbatches(
            model, train_dataloader, loss_fn, optimizer_, train_steps
        )

        metric_dict, test_loss = evalbatches(
            model, test_dataloader, loss_fn, metrics, eval_steps
        )

        scheduler.step(test_loss)

        if tunewriter:
            tune.report(
                iterations=epoch,
                train_loss=train_loss,
                test_loss=test_loss,
                **metric_dict,
            )
        else:
            writer.add_scalar("Loss/train", train_loss, epoch)
            writer.add_scalar("Loss/test", test_loss, epoch)
            for m in metric_dict:
                writer.add_scalar(f"metric/{m}", metric_dict[m], epoch)
            lr = [group["lr"] for group in optimizer_.param_groups][0]
            writer.add_scalar("learning_rate", lr, epoch)
            metric_scores = [f"{v:.4f}" for v in metric_dict.values()]
            logger.info(
                f"Epoch {epoch} train {train_loss:.4f} test {test_loss:.4f} metric {metric_scores}"  # noqa E501
            )

    return model

In [5]:
## uit train_model.py

def trainbatches(
    model: GenericModel,
    traindatastreamer: Iterator,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    train_steps: int,
) -> float:
    model.train()
    train_loss: float = 0.0
    for _ in tqdm(range(train_steps)):
        x, y = next(iter(traindatastreamer))
        optimizer.zero_grad()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.detach().numpy()
    train_loss /= train_steps
    return train_loss


## uit train_model.py
def evalbatches(
    model: GenericModel,
    testdatastreamer: Iterator,
    loss_fn: Callable,
    metrics: List[Metric],
    eval_steps: int,
) -> Tuple[Dict[str, float], float]:
    model.eval()
    test_loss: float = 0.0
    metric_dict: Dict[str, float] = {}
    for _ in range(eval_steps):
        x, y = next(iter(testdatastreamer))
        yhat = model(x)
        test_loss += loss_fn(yhat, y).detach().numpy()
        for m in metrics:
            metric_dict[str(m)] = (
                metric_dict.get(str(m), 0.0) + m(y, yhat).detach().numpy()
            )

    test_loss /= eval_steps
    for key in metric_dict:
        metric_dict[str(key)] = metric_dict[str(key)] / eval_steps
    return metric_dict, test_loss

In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [7]:
## Model 1: CNN_een met een convolutional layer & drie linear layers

@gin.configurable
class CNN_1_3(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(6272, 3136),
            nn.ReLU(),
            nn.Linear(3136, 1568),
            nn.ReLU(),
            nn.Linear(1568, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [8]:
## Model 2.2: CNN_twee met twee convolutional layers & twee linear layers

@gin.configurable
class CNN_2_2(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [9]:
## Model 2.2.16.32: CNN_twee met twee convolutional layers & twee linear layers & 16-32 filters

@gin.configurable
class CNN_2_2_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits



In [10]:
## Model 2.2.32.64: CNN_twee met twee convolutional layers & twee linear layers & 32-64 filters

@gin.configurable
class CNN_2_2_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits



In [ ]:
## Model 2.2.64.128: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_2_2_64_128(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4608, 2304),
            nn.ReLU(),
            nn.Linear(2304, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [ ]:
## Model 2.2.128.256: CNN_twee met twee convolutional layers & twee linear layers & 128-256 filters

@gin.configurable
class CNN_2_2_128_256(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216, 4608),
            nn.ReLU(),
            nn.Linear(4608, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [11]:
## Model 2.3: CNN_twee met twee convolutional layers en drie linear layers

@gin.configurable
class CNN_2_3(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [12]:
## Model 2.3.16.32: CNN_twee met twee convolutional layers en drie linear layers

@gin.configurable
class CNN_2_3_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [13]:
## Model 2.3.32.64: CNN_twee met twee convolutional layers en drie linear layers

@gin.configurable
class CNN_2_3_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [14]:
## Model 2.4: CNN_twee met twee convolutional layers & vier linear layers

@gin.configurable
class CNN_2_4(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, 144),
            nn.ReLU(),
            nn.Linear(144, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [15]:
## Model 3.2: CNN_drie met twee convolutional layers & twee linear layers

@gin.configurable
class CNN_3_2(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [16]:
## Model 3.3: CNN_drie met drie convolutional layers

@gin.configurable
class CNN_3_3(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [17]:
## Model 3.4: CNN_drie met drie convolutional layers & vier linear layers

@gin.configurable
class CNN_3_4(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [ ]:
## Model 3.4.64.128: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_3_4_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [ ]:
## Model 3.4.64.128: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_3_4_64_128(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4608, 2304),
            nn.ReLU(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [ ]:
## Model 3.4.128.256: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_3_4_128_256(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216, 4608),
            nn.ReLU(),
            nn.Linear(4608, 2304),
            nn.ReLU(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [18]:
gin.parse_config_file("model.gin")

ParsedConfigFileIncludesAndImports(filename='model.gin', imports=['gin.torch.external_configurables'], includes=[])

In [ ]:
print(summary(model))
print(gin.config_str())

In [27]:
datadir =  Path("/home/admindme/code/ML22_opdracht1/data/raw/FashionMNIST")
train_dataloader, test_dataloader = get_MNIST(datadir, 64)
len(train_dataloader), len(test_dataloader)
accuracy = Accuracy()

In [36]:
## Stap 1 
model = CNN_1_3().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 15:51:35.289 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-1551
100%|██████████| 938/938 [03:52<00:00,  4.04it/s]
2022-12-23 15:55:32.359 | INFO     | __main__:trainloop:68 - Epoch 0 train 43.8874 test 2.3037 metric ['0.1069']
100%|██████████| 938/938 [05:02<00:00,  3.10it/s]
2022-12-23 16:00:39.257 | INFO     | __main__:trainloop:68 - Epoch 1 train 2.3066 test 2.3079 metric ['0.0950']
100%|██████████| 938/938 [05:02<00:00,  3.10it/s]
2022-12-23 16:05:46.406 | INFO     | __main__:trainloop:68 - Epoch 2 train 2.3071 test 2.3080 metric ['0.0979']
100%|██████████| 3/3 [14:10<00:00, 283.63s/it]


In [31]:
## Stap 1 
model = CNN_2_3().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 15:37:20.023 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-1537
100%|██████████| 938/938 [00:28<00:00, 32.52it/s]
2022-12-23 15:37:51.203 | INFO     | __main__:trainloop:68 - Epoch 0 train 2.5855 test 2.3057 metric ['0.1016']
100%|██████████| 938/938 [00:31<00:00, 29.76it/s]
2022-12-23 15:38:24.963 | INFO     | __main__:trainloop:68 - Epoch 1 train 2.3072 test 2.3051 metric ['0.0990']
100%|██████████| 938/938 [00:31<00:00, 30.20it/s]
2022-12-23 15:38:58.191 | INFO     | __main__:trainloop:68 - Epoch 2 train 11.2988 test 2.3144 metric ['0.0973']
100%|██████████| 3/3 [01:38<00:00, 32.67s/it]


In [ ]:
## Stap 1 
model = CNN_3_3().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [30]:
## Stap 2 
model = CNN_2_2().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 15:35:18.307 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-1535
100%|██████████| 938/938 [00:26<00:00, 34.82it/s]
2022-12-23 15:35:47.489 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.8262 test 0.6119 metric ['0.7674']
100%|██████████| 938/938 [00:28<00:00, 32.70it/s]
2022-12-23 15:36:18.307 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.5509 test 0.5754 metric ['0.7936']
100%|██████████| 938/938 [00:29<00:00, 32.28it/s]
2022-12-23 15:36:49.404 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.5486 test 0.5939 metric ['0.7932']
100%|██████████| 3/3 [01:30<00:00, 30.31s/it]


In [32]:
## Stap 2
model = CNN_2_4().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 15:38:58.316 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-1538
100%|██████████| 938/938 [00:29<00:00, 31.87it/s]
2022-12-23 15:39:29.986 | INFO     | __main__:trainloop:68 - Epoch 0 train 3.0085 test 2.3056 metric ['0.1031']
100%|██████████| 938/938 [00:31<00:00, 29.38it/s]
2022-12-23 15:40:03.943 | INFO     | __main__:trainloop:68 - Epoch 1 train 2.3072 test 2.3043 metric ['0.0973']
100%|██████████| 938/938 [00:31<00:00, 29.50it/s]
2022-12-23 15:40:37.812 | INFO     | __main__:trainloop:68 - Epoch 2 train 2.3071 test 2.3078 metric ['0.1023']
100%|██████████| 3/3 [01:39<00:00, 33.11s/it]


In [33]:
## Stap 2
model = CNN_3_2().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 15:40:37.923 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-1540
100%|██████████| 938/938 [00:25<00:00, 36.56it/s]
2022-12-23 15:41:05.749 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.8298 test 0.7018 metric ['0.7391']
100%|██████████| 938/938 [00:25<00:00, 36.91it/s]
2022-12-23 15:41:33.243 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.6579 test 0.6638 metric ['0.7675']
100%|██████████| 938/938 [00:25<00:00, 36.72it/s]
2022-12-23 15:42:00.815 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.6492 test 0.6709 metric ['0.7474']
100%|██████████| 3/3 [01:22<00:00, 27.58s/it]


In [35]:
## Stap 2
model = CNN_3_4().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 15:43:23.387 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-1543
100%|██████████| 938/938 [00:25<00:00, 37.36it/s]
2022-12-23 15:43:50.707 | INFO     | __main__:trainloop:68 - Epoch 0 train 2.3202 test 2.3060 metric ['0.0968']
100%|██████████| 938/938 [00:25<00:00, 36.57it/s]
2022-12-23 15:44:18.445 | INFO     | __main__:trainloop:68 - Epoch 1 train 2.3070 test 2.3090 metric ['0.0979']
100%|██████████| 938/938 [00:25<00:00, 36.82it/s]
2022-12-23 15:44:45.963 | INFO     | __main__:trainloop:68 - Epoch 2 train 2.3072 test 2.3072 metric ['0.0982']
100%|██████████| 3/3 [01:22<00:00, 27.47s/it]


In [34]:
## Stap 3
model = CNN_2_2_16_32().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 15:42:00.902 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-1542
100%|██████████| 938/938 [00:25<00:00, 37.01it/s]
2022-12-23 15:42:28.570 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.8986 test 0.7151 metric ['0.7539']
100%|██████████| 938/938 [00:25<00:00, 37.17it/s]
2022-12-23 15:42:55.920 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.7129 test 0.7403 metric ['0.7481']
100%|██████████| 938/938 [00:25<00:00, 37.19it/s]
2022-12-23 15:43:23.275 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.6766 test 0.7022 metric ['0.7561']
100%|██████████| 3/3 [01:22<00:00, 27.40s/it]


In [ ]:
## Stap 3
model = CNN_2_2_32_64().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [ ]:
## Stap 3
model = CNN_2_3_16_32().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [ ]:
## Stap 3
model = CNN_2_3_32_64().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [ ]:
## stap 3 Nieuw
model = CNN_2_2_64_128().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [ ]:
## Stap 3 Nieuw
model = CNN_2_2_128_256().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )